In [2]:
import plotly.plotly as py
import plotly.graph_objs as go      

import pandas as pd
import math
import numpy as np

import statsmodels
import scipy as sc
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('experiment.csv')

In [4]:
print data

    index  feature1  feature2
0       1      0.07      0.27
1       2      0.12      0.35
2       3      0.20      0.45
3       4      0.27      0.52
4       5      0.32      0.57
5       6      0.37      0.61
6       7      0.42      0.65
7       8      0.47      0.71
8       9      0.55      0.73
9      10      0.65      0.73
10     11      0.17      0.38
11     12      0.65      0.77
12     13      0.55      0.75


Удалим 9 и 10 измерения в силу выброса, далее это обсудим подробнее 

feature1 = $m,~g$

feature2 = $U_{isu},~B$

feature3 = $F,~g \cdot cm \cdot s^{-2}$ = feature1 $\cdot 10^{3}$

feature4 = $U_{isu}^2,~B^2$

feature5 = $2 d \sqrt{\frac{2 \pi F}{S}},~g^{1/2} {cm}^{1/2} s^{-1}~=~U_{cgs}$

feature6 = $\sigma_F^{syst}~=~0.03 \cdot feature3$ 

feature7 = $\sigma_{U^2}^{syst}~=~2U \sigma^{syst}_u $

feature8 = $\sigma_{U_{cgs}}^{syst}$

In [5]:
d = 0.16
sigma_syst_d = 0.01
s = 42 
sigma_syst_s = 1
epsilon_syst_m = 0.03
sigma_syst_u = 0.02

In [6]:
data['feature2'] = data['feature2']*1000

In [7]:
data.insert(3, 'feature3', data['feature1']*1000)

In [8]:
data.insert(4, 'feature4', data['feature2']*data['feature2'])

In [30]:
print data

    index  feature1  feature2  feature3  feature4  feature5  feature6  \
0       1      0.07       270        70     72900  1.035534       2.1   
1       2      0.12       350       120    122500  1.355832       3.6   
2       3      0.20       450       200    202500  1.750372       6.0   
3       4      0.27       520       270    270400  2.033748       8.1   
4       5      0.32       570       320    324900  2.214064       9.6   
5       6      0.37       610       370    372100  2.380763      11.1   
6       7      0.42       650       420    422500  2.536529      12.6   
7       8      0.47       710       470    504100  2.683269      14.1   
10     11      0.17       380       170    144400  1.613763       5.1   
11     12      0.65       770       650    592900  3.155527      19.5   
12     13      0.55       750       550    562500  2.902663      16.5   

    feature7  feature8  empty  
0      10800  0.069344      1  
1      14000  0.088924      1  
2      18000  0.113015     

In [10]:
data.insert(5, 'feature5', data['feature3'].apply(np.sqrt)*2*d*math.sqrt(2*math.pi/s))

In [11]:
data.insert(6, 'feature6', data['feature3']*epsilon_syst_m)

In [12]:
data.insert(7, 'feature7', sigma_syst_u*1000 * 2 * data['feature2'])

In [13]:
data.insert(8, 'feature8', data['feature5']*(1/d*sigma_syst_d + 1/(data['feature3']+s)*0.5))

In [17]:
data.insert(9, 'empty', 1)

In [14]:
data = data.drop([9,8])

Регрессия

In [41]:
m1 = smf.ols('feature4 ~ feature3', 
             data=data)
fitted = m1.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:               feature4   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                     571.9
Date:                Thu, 08 Sep 2016   Prob (F-statistic):           1.87e-09
Time:                        23:52:11   Log-Likelihood:                -125.32
No. Observations:                  11   AIC:                             254.6
Df Residuals:                       9   BIC:                             255.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   6156.5472   1.52e+04      0.405      0.6

ValueError: shapes (11,2) and (11,) not aligned: 2 (dim 1) != 11 (dim 0)

In [35]:
m2 = smf.ols('feature2 ~ feature5', 
             data=data)
fitted = m2.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:               feature2   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     832.4
Date:                Thu, 08 Sep 2016   Prob (F-statistic):           3.52e-10
Time:                        23:49:36   Log-Likelihood:                -46.566
No. Observations:                  11   AIC:                             97.13
Df Residuals:                       9   BIC:                             97.93
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.6800     19.708      0.136      0.8

In [70]:
data_err = pd.DataFrame()
data_err.insert(0, 'err', (data['feature4']-976.1712*data['feature3'])/data['feature4'])

In [71]:
print data_err

         err
0   0.062661
1   0.043751
2   0.035880
3   0.025273
4   0.038551
5   0.029338
6   0.029605
7   0.089862
10 -0.149232
11 -0.070183
12  0.045521


In [72]:
stats.probplot(data_err, dist = "norm", plot = pylab)
pylab.show()

NameError: name 'stats' is not defined

array([[ 0.0626614 ],
       [ 0.04375066],
       [ 0.0358803 ],
       [ 0.02527284],
       [ 0.03855099],
       [ 0.02933796],
       [ 0.02960496],
       [ 0.0898622 ],
       [-0.14923202],
       [-0.07018263],
       [ 0.04552149]])